#### CSCE 489 :: Recommender Systems :: Texas A&M University :: Spring 2021


# Homework 1: Non-personalized Recommendations

### 100 points [10% of your final grade]

- **Due Wednesday, February 10 by 11:59pm**

*Goals of this homework:* The objective of this homework is to get you familiar with the pipelines of explicit and implicit recommendation, including data processing, building non-personalized recommenders for both explicit and implicit scenarios, and evaluating the  results.

*Submission instructions (Canvas):* To submit your homework, rename this notebook as `UIN_hw1.ipynb`. For example, if your UIN is `123456789`, then your homework submission would be `123456789_hw1.ipynb`. Submit this notebook via Canvas (look for homework 1 under the assignment section). Your notebook should be completely self-contained, with the results visible in the notebook. We should not have to run any code from the command line, nor should we have to run your code within the notebook (though we reserve the right to do so). So please run all the cells for us, and then submit. 

*Late policy:* You may use up to three of your late days on this assignment. No homeworks will be accepted after February 13 11:59pm.

## Collaboration Declaration:

***You must add all of your collaboration declarations here. Who did you talk to about this assignment? What web resources did you use? Etc.***

For example:
* Part 1a: I talked to Amy about how to split the data randomly. She helped me understand that I needed to use a random number generator.
* Part 1b: I needed help on how to comment my code, so I relied on this StackOverflow thread: https://stackoverflow.com/questions/184618/what-is-the-best-comment-in-source-code-you-have-ever-encountered
* I used the microsoft teams postings to compare what I was reporting and what others were reporting.
* I also used pandas documentation to help split the data into 70:30 https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html 


# Part 1. Recommendations with User Ratings (Explicit Feedback) (50 points total)

In this first part, we're going to build a non-personalized recommender based on user ratings.  In many online platforms, such as Amazon, IMDb, and MovieLens, users are able to express their preference to items by explicit ratings (like by assigning a 1-5 star rating to a movie). We're going to use those ratings to generate a recommendation. For this homework, we're focusing on **non-personalized** recommendations (that is, everyone gets the same recommendation). In our future homeworks, we'll take a look at how to personalize these recommendations so each user gets a different recommended list.

For this part, we will:

* load and process the MovieLens 1M dataset, 
* build the non-personalized recommender, and 
* evaluate the recommender.

## Part 1a: Load and process the data (20 points)

To start out, we need to prepare the data. We will use the MovieLens 1M data from https://grouplens.org/datasets/movielens/1m/ in this homework. Lucky for you, we are providing the file containing the ratings -- ratings.dat  -- so all you need to do is load the ratings.dat file in the notebook as a DataFrame variable using the Pandas library. The code to do this has been provided in the next cell, but you need to run it. The DataFrame variable is named as 'data_df'. The code names the column of movie id as 'MovieID', names the column of user id as 'UserID', names the column of rating as 'Rating', and the column of timestamp as 'Timestamp'. After this, the first 5 rows of data_df are printed out by DataFrame.head().

Note that you should install the pandas library before you use it. For more information about Pandas, here is a quick introduction: https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html. 

In [1]:
import pandas as pd

data_df = pd.read_csv('./ratings.dat', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"], engine="python")
data_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


Now let's find some simple statistics for this dataset. In the next cell, please count and print how many unique users, unique movies, and how many ratings there are in this dataset. 

Hint: You may need to use the function pandas.Series.unique().

In [2]:
# count and print how many unique users, unique movies, and ratings in this dataset
# Unique Users:
print("Unique Users: " + str(data_df['UserID'].unique().size))
#Unique Movies:
print("Unique Movies: " + str(data_df['MovieID'].unique().size))
#Unique Ratings:
print("Unique Ratings: " + str(len(data_df["Rating"])))

Unique Users: 6040
Unique Movies: 3706
Unique Ratings: 1000209


Because in Python, the index for a list starts from 0, it is more convenient if we have the ids of users and movies start from 0 as well. Moreover, we also need to make sure the UserID and MovieID are continuous, so in the next cell, we reindex UserID and MovieID. The code is already provided, but you still need to run it and it is a good chance for you to check the correctness of your previous code by running the provided code. (It may take a few minutes to run it.)

In [3]:
# First, generate dictionaries for mapping old id to new id for users and movies
unique_MovieID = data_df['MovieID'].unique()
unique_UserID = data_df['UserID'].unique()
j = 0
user_old2new_id_dict = dict()
for u in unique_UserID:
    user_old2new_id_dict[u] = j
    j += 1
j = 0
movie_old2new_id_dict = dict()
for i in unique_MovieID:
    movie_old2new_id_dict[i] = j
    j += 1
    
# Then, use the generated dictionaries to reindex UserID and MovieID in the data_df
for j in range(len(data_df)):
    data_df.at[j, 'UserID'] = user_old2new_id_dict[data_df.at[j, 'UserID']]
    data_df.at[j, 'MovieID'] = movie_old2new_id_dict[data_df.at[j, 'MovieID']]

Now, you have got a ready-to-use dataset. The next step is to split it **randomly** into training and testing sets so that you can build your recommendation model based on the training set and evaluate it on the testing set. Here you need to split the data_df into two parts: a DataFrame train_df containing 70% user-movie-rating samples in data_df, and a DataFrame test_df containing 30% samples. train_df and test_df should have no overlap. In the next cell, write your code and print the numbers of samples in the generated train_df and test_df at last.

Hint: you may need to use functions from numpy.random for generating random numbers.

In [4]:
import numpy as np
train_df = data_df.sample(frac=0.7, random_state=1)
#train_df.he
test_df = data_df.drop(train_df.index)
print("train_df entries: " + str(len(train_df)))
print("test_df entries: " + str(len(test_df)))

train_df entries: 700146
test_df entries: 300063


Last, we need to generate numpy array variables (i.e., matrix version of dataset) for both train_df and test_df for the ease of calculation in the next step. More specifically, we will generate two numpy array variables of size (#user, #movie) with each entry representing the user-movie rating. And if the user-movie rating is missing, then the corresponding entry is 0. The code is already provided, but you still need to run it and it is a good chance for you to check the correctness of your previous code by running the provided code. Please make sure you have already installed the scipy library before running the code.

In [5]:
from scipy.sparse import coo_matrix

num_user = len(data_df['UserID'].unique())
num_movie = len(data_df['MovieID'].unique())

train_mat = coo_matrix((train_df['Rating'].values, (train_df['UserID'].values, train_df['MovieID'].values)), shape=(num_user, num_movie)).toarray().astype(float)
test_mat = coo_matrix((test_df['Rating'].values, (test_df['UserID'].values, test_df['MovieID'].values)), shape=(num_user, num_movie)).toarray().astype(float)

## Part 1b: Build the non-personalized recommender (15 points)

In this part, you need to build a non-personalized recommendation model to predict user-movie ratings. The model is very simple: for each movie, you calculate the average rating of this movie in the training dataset, and use this average rating as the prediction for all users with respect to this movie. In this way, the prediction will be the same across all users, i.e., it is non-personalized. In the next cell, you need to write your code to implement this algorithm, and generate a numpy array variable named 'prediction_mat' of size (#user, #movie) with each entry showing the predicted rating for corresponding user-movie pair.

* Hint1: the prediction_mat can only be calculated from train_mat, you cannot use test_mat here. 
* Hint2: the average rating should be calculated only from ratings (1-5) without considering the missing entries (0) in train_mat. 
* Hint3: if there are movies without any rating in train_mat, set the prediction for these movies as the average rating of the whole train_mat.


In [6]:
prediction_mat = np.empty([6040, 3706])#let's create an empty numpy array for prediction_mat
running_avg_all = 0 #set the running avg to 0
total_ratings = 0 #set the number of ratings to 0, although I probably don't have to track this.

for movie in range(len(train_mat[0])): #for every movie as we iterate up
    avg_rating = 0 #set/reset avg_rating for a movie to 0 in the begining of every loop
    num_ref = 0 # set/reset the number of ratings are given at the begining of every loop
    
    for user in range(len(train_mat)): #for every movie as we iterate across
        if(train_mat[user][movie] > 0 and train_mat[user][movie] < 6): 
            #if the points are greater than 0 or less than 6, then it was rated
            avg_rating += train_mat[user][movie] #add the value in the array to the average rating
            num_ref += 1 #update the number of ratings
            running_avg_all += train_mat[user][movie] #do the same for the running average
            total_ratings +=1 #update total amt. of ratings that have been given.
            #print("avg_rating" + str(avg_rating))
    
    #after we have run through all the users for that movie
    if(avg_rating != 0): #as long as their is a rating for that movie...
        avg_rating /= num_ref #divide average rating by number of references to that movie

    for user in range(len(prediction_mat)): #set out the value across the matrix for all users
        prediction_mat[user][movie] = avg_rating


running_avg_all /= total_ratings #now let's find the overall average

#I strayed from the user, row convention because this was pretty simple and it checks all the values anyways
for i in range(len(prediction_mat)): 
    for j in range(len(prediction_mat[0])):
        if(prediction_mat[i][j] == 0): #if there is any value that is 0, set it to the overall average
            prediction_mat[i][j] = running_avg_all


Please print out the id of the top-5 movies with largest predicted ratings and their predicted ratings.

In [7]:
highest_rated = [0,0,0,0,0] #made an array that has all the highest ratings
highest_rated_indexes = [0,0,0,0,0] #this array will hold the MovieID of the highest ratings

for i in range(len(prediction_mat[0])): #for every movie within the array
    if(prediction_mat[0][i] >= highest_rated[0]): #if the value is higher than the current highest value..
        highest_rated[4] = highest_rated[3] # each of these values will shift down one, that way the highest
        highest_rated[3] = highest_rated[2] #at one point in the list will probably shift through
        highest_rated[2] = highest_rated[1] #the entire list. 
        highest_rated[1] = highest_rated[0]
        highest_rated[0] = prediction_mat[0][i]
       
        highest_rated_indexes[4] = highest_rated_indexes[3] #the same general algorithm is used in passing 
        highest_rated_indexes[3] = highest_rated_indexes[2] #down the MovieIDs of each of the top ratings
        highest_rated_indexes[2] = highest_rated_indexes[1] #that way, there is a unity between the matrixes
        highest_rated_indexes[1] = highest_rated_indexes[0]
        highest_rated_indexes[0] = i
#print out the title...
print("The top 5 movies and their ratings are: ")
for i in range(len(highest_rated)): #for all the information
    print("MovieID: " + str(highest_rated_indexes[i]) +" , rated at: "+ str(highest_rated[i]))#print out id and rating


The top 5 movies and their ratings are: 
MovieID: 3704 , rated at: 5.0
MovieID: 3697 , rated at: 5.0
MovieID: 3683 , rated at: 5.0
MovieID: 3638 , rated at: 5.0
MovieID: 3629 , rated at: 5.0


## Part 1c: Evaluate the non-personalized recommender (15 points)

In this part, you need to evaluate your prediction over the held-out testing dataset test_mat. For explicit recommendation, a typical metric is Root Mean Square Error (RMSE). In the next cell, you need to write your code to calculate the RMSE with your prediction_mat and the test_mat, and print it out.


In [8]:
#calculate RMSE
running_diff = 0 # it is the sum of the differences across all points
for i in range(len(prediction_mat)):
    for j in range(len(prediction_mat[0])): #for every point essentially as it iterates over each user
        running_diff += (prediction_mat[i][j] - test_mat[i][j])**2 #add the difference between the prediction 
        #and test matrices squared.
#after finding that total sum of differences squared
rmse = running_diff / (len(prediction_mat) * len(prediction_mat[0])) #divide by all data points
#my initial thought was to have only unique reviews dividing the value, however since we filled in the 0s
#with the running average there would be no "un-reviewed" values.
rmse = rmse**(0.5) #apply the square root to the value
print("RMSE: "+str(rmse))

RMSE: 3.2952220280126445


# Part 2. Recommendations with implicit feedback (50 points total)

In many scenarios, we may not have explcit ratings. But we often have lots of implicit feedback. For this part, we're going to build a simple non-personalized implicit recommendation algorithm. Since feedback like user clicks, purchases, and views is much more widespread than explicit ratings, implicit recommenders offer great opportunities for far-reaching impact. 

Concretely, the task of implicit recommendation is to recommend items to users based on implicit signals from users, i.e., we only know what items a user is interested in, but have no idea what items the user dislikes. So for this case, the dataset we could use for this implicit recommendation experiment only contains binary data with 1 representing that the user likes the item, and with 0 representing that we don't know the user's preference towards the item. Because of this, we cannot use the same evaluation method as explicit recommendation. Instead, we need to evaluate the implicit recommendation quality by a ranking task.

In this part, you will use the same MovieLens 1M dataset, and:
* write the code to transfer the explicit dataset to implicit one,
* build a non-personalized implicit recommender, 
* and evaluate your recommender.

## Part 2a: Process the data (10 points)

It is very easy to transfer the explicit datasets you already generated to implicit ones: here, you will consider the watching behavior as the implicit signal showing that the user is interested in a movie. Thus, you can use the same train_df and test_df for implicit recommendation experiment with 'Rating' column ignored. And for train_mat and test_mat, you need to make all ratings to be value 1 and keep 0 entries the same. Write your code to transfer the train_mat and test_mat.

In [9]:
# Your Code Here...
for i in range(len(train_mat)): #for all the cols in train mat
    for j in range(len(train_mat[0])): #for all the rows in those cols
        if(train_mat[i][j] > 0): #if it's rated
            train_mat[i][j] = 1 #set it to 1
        else:
            train_mat[i][j] = 0 #set to 0

for i in range(len(test_mat)): #for all the cols in test mat
    for j in range(len(test_mat[0])): #for all the rows in those cols
        if(test_mat[i][j] > 0): #if it's rated
            test_mat[i][j] = 1 #set to 1
        else:
            test_mat[i][j] = 0 #set to 0


## Part 2b: Build the non-personalized recommender (20 points)

In this part, you need to build a non-personalized recommendation model to provide a ranked list of 50 movies as the recommendation for each user. The model is very simple: for each user, the recommendation list is to rank the unwatched movies by their **popularity**, where the popularity is the number of implicit feedback each movie gets. In this case, although it is non-personalized recommender, the recommendation results may be different for users because the unwatched movies are different across users.

In the next cell, write your code to generate the ranked lists of movies by the popularity based recommendation algorithm for every user, store the result in a numpy array of size (#user, 50), entry (u, k) represents the movie id that is ranked at position k in the recommendation list to user u. Print out the top-5 recommended movies and their popularity for the first user (with id 0).

Hint: the popularity can only be calculated from train_mat, you cannot use test_mat here. 

In [10]:
user_rec_mat = np.empty([6040, 50]) #make a matrix that will accomodate all the users
movies_in_ranking = np.empty([3706,2]) # make a ranked list of all the movies

for movie in range(len(train_mat[0])):#for every movie
    movie_pop = 0 #(movie popularity), how many people have seen it
    for user in range(len(train_mat)): #for every user rating for that movie
        movie_pop += train_mat[user][movie] #add the values, will show how many views.
    #once the popularity of that movie is in
    movies_in_ranking[movie] = [movie, movie_pop]
    #starts from 0, and iterates through to the end, and has the corresponding ranking popularity

movies_in_ranking = sorted(movies_in_ranking, key=lambda x: x[1], reverse=True) 
#I want to sort this movie  list by the ranking, that way I can just pull from the first missing 50 
#movies from each user instead of iterating through every person and figuring out ranking etc. 

for user in range(len(train_mat)): #for every user in the training matrix
    num_ranked = 0 #this will act as our first 50 index
    #we'll iterate through the first x amt of items in the ranked list.
    for movie in range(len(movies_in_ranking)):
        movie_index = int(movies_in_ranking[movie][0])
        if(num_ranked == 50): #if we have 50 entries into our recomendation list for that user, move to the next user
            break
        if(train_mat[user][movie_index] == 0): #if the user hasn't seen that movie
            user_rec_mat[user][num_ranked] = movies_in_ranking[movie][0] 
            #user_rec_mat at user and recomendation # is equal to the index of the movie next in the list
            num_ranked += 1 #update the index

print("The first 5 recommendations for user 0 is: ")
for i in range(0,5): #printing out the first 5 recomendations for user 0
    print(user_rec_mat[0][i])


The first 5 recommendations for user 0 is: 
104.0
124.0
64.0
97.0
113.0


## Part 2c: Evaluate the non-personalized recommender (20 points)

In this part, you need to evaluate your non-personalized recommendation by the held-out testing dataset test_mat for each user. For the implicit recommendation, two typical metrics are recall@k and precision@k. Here, you need to write the code to calculate recall@k and  precision@k for k=5, 20, 50 for each user, i.e., six metrics for every user. And please print out the average over all users for these six metrics.

Hint: if a user does not have any testing samples in test_mat, do not include this user in the final averaged metric.


In [11]:
def precision_at_k(rec_movies, test_mat, k, user): #A function will be easier to call
    #precision@K = good recommendations / k 
    good_recs = 0 
    #if the recommended movies is present in the train set for k number of recs
    
    for i in range(0,k):
        if( int(test_mat[user][int(rec_movies[user][i])]) == 1): #if what was recomended matches with the test set
            good_recs += 1; #increment good recommendations
            
    return float(good_recs/k)#return good recommendations / k

def recall_at_k(rec_movies, test_mat, k, user):
    #recall@K = good recommendations / 
    numerator = 0
    denom = 0
    
    for i in range(0,k):
        if(int(test_mat[user][int(rec_movies[user][i])]) == 1):#if it's a good rec, increment the numerator
            numerator += 1
    
    for i in range(len(test_mat[0])): #if there is a click for that user increment their denominator
        if(test_mat[user][i] == 1):
            denom += 1;
    
    return float(numerator/denom);

#p = precision, a = @, the number = k value
pa5 = 0
pa20 = 0
pa50 = 0
ra5 = 0
ra20 = 0
ra50 = 0

for i in range(len(test_mat)): #for all users add up each respective k computation
    pa5 += precision_at_k(user_rec_mat, test_mat, 5, i)
    pa20 += precision_at_k(user_rec_mat, test_mat, 20, i)
    pa50 += precision_at_k(user_rec_mat, test_mat, 50, i)
    ra5 += recall_at_k(user_rec_mat, test_mat, 5, i)
    ra20 += recall_at_k(user_rec_mat, test_mat, 20, i)
    ra50 += recall_at_k(user_rec_mat, test_mat, 50, i)
    #number of recs liked, divided by the total amt of recs
#calculate the averages
pa5 /= len(test_mat)
pa20 /= len(test_mat)
pa50 /= len(test_mat)
ra5 /= len(test_mat)
ra20 /= len(test_mat)
ra50 /= len(test_mat)
#printing the results
print("avg Precision@5: " + str(pa5))
print("avg Precision@20: " + str(pa20))
print("avg Precision@50: " + str(pa50))
print("avg Recall @ 5: " + str(ra5))
print("avg Recall @ 20: " + str(pa20))
print("avg Recall @ 50: " + str(pa50))

avg Precision@5: 0.27033112582782204
avg Precision@20: 0.2108692052980102
avg Precision@50: 0.15764569536423548
avg Recall @ 5: 0.036770203862371585
avg Recall @ 20: 0.2108692052980102
avg Recall @ 50: 0.15764569536423548
